In [3]:
import pandas as pd
import numpy as np

In [4]:
transcriptsv1 = pd.read_csv("transcripts_dataset_v1.csv.gz", compression="gzip")
transcriptsv2 = pd.read_csv("transcripts_dataset_v2.csv.gz", compression="gzip")
metadata = pd.read_csv("metadata.csv.gz", compression='gzip')

In [5]:
print('Transcipt v1 shape:', transcriptsv1.shape)
print('Transcipt v2 shape:', transcriptsv2.shape)
print('Metadata shape:', metadata.shape, '\n')

# print(list(transcriptsv1.columns))
# print(list(transcriptsv2.columns))
# print(list(metadata.columns))

# print('\n')

# print('Transcript v1 null check:\n', transcriptsv1.isna().sum(), '\n')
# print('Transcript v2 null check:\n', transcriptsv2.isna().sum(), '\n')
print('Metadata null check:\n', metadata.isna().sum()) # show_name 2 nulls, episode_description 205 nulls, category 6000, pubdate 20030


Transcipt v1 shape: (39892, 6)
Transcipt v2 shape: (65468, 6)
Metadata shape: (105360, 14) 

Metadata null check:
 show_uri                       0
show_name                      0
show_description               2
publisher                      0
language                       0
rss_link                       0
episode_uri                    0
episode_name                   0
episode_description          205
duration                       0
show_filename_prefix           0
episode_filename_prefix        0
category                    6000
pubdate                    20030
dtype: int64


In [6]:
# concatenate transcripts dataframes
transcripts = pd.concat([transcriptsv1, transcriptsv2])
print(transcripts.shape)

# subset metadata
metadata = metadata[["show_name", "show_description", "publisher", "language", "episode_name", "episode_description", "duration", "show_filename_prefix", "episode_filename_prefix", 'category', 'pubdate']]
metadata = metadata.rename({"episode_filename_prefix": "episode_id", "show_filename_prefix": "show_id"}, axis="columns")  # rename cols

# remove episode_id suffix 
transcripts.episode_id = transcripts.episode_id.apply(lambda x: x.replace(".json", ""))
transcripts = transcripts.drop("Unnamed: 0", axis=1)

# remove whitespace
metadata.episode_id = metadata.episode_id.apply(lambda x: x.strip())

(105360, 6)


In [7]:
# join data
full_dataset = transcripts.join(metadata.set_index("episode_id"), on="episode_id", rsuffix="_trans")
full_dataset.isna().sum()

show_id                    0
episode_id                 0
transcript                 0
avg_confidence             0
word_count                 0
show_name                  0
show_description           2
publisher                  0
language                   0
episode_name               0
episode_description      205
duration                   0
show_id_trans              0
category                6000
pubdate                20030
dtype: int64

In [8]:
print(full_dataset.shape)
# full_dataset.head(5)

(105360, 15)


In [9]:
# Null inspection
full_dataset[full_dataset.isnull().any(axis=1)] # metadata discrepancies, episode_id present for all instances

,show_id,episode_id,transcript,avg_confidence,word_count,show_name,show_description,publisher,language,episode_name,episode_description,duration,show_id_trans,category,pubdate
5,show_2uf4dRlrtFgANIXtmsNS7H,22Hee4IumtEZ3PiHp21m8V,Hayward we did it step one done. Hello. Welcom...,0.834999,17174,The Reng,The funniest thing you’ve ever heard- from ver...,Alex Rengers,['en'],The Reng Podcast |Ep. 1|,"First ever podcast by you'rs truly, Alex Renge...",24.119733,show_2uf4dRlrtFgANIXtmsNS7H,NaN,NaN
7,show_2UyVUlVZBrZxOPcNeHO59D,3uZtH30AGm11sJEsBEZNqm,Hello and welcome to combos with Chloe. Today....,0.829958,15372,"Convos With Chloe: God, Faith, Dating & Relati...",Why are relationships so hard? Why do I worry ...,Chloe M. Gooden,['en'],When It Feels Like God Isn't There,Have you ever felt like God wasn't with you in...,15.790733,show_2UyVUlVZBrZxOPcNeHO59D,Religion & Spirituality,NaN
8,show_2UyVUlVZBrZxOPcNeHO59D,7yME8WJllFMuj7rbBtFc5t,Hello and welcome to combos with Chloe. So tod...,0.823783,23793,"Convos With Chloe: God, Faith, Dating & Relati...",Why are relationships so hard? Why do I worry ...,Chloe M. Gooden,['en'],"When He Says ""He Isn't Ready for a Relationshi...",Ugh! That frustrating sentence that we all hat...,22.637900,show_2UyVUlVZBrZxOPcNeHO59D,Religion & Spirituality,NaN
9,show_2UyVUlVZBrZxOPcNeHO59D,0jSpyAIsU7R8BQ5rP5EUgy,Hello and welcome to combos with Chloe. So tod...,0.824384,21046,"Convos With Chloe: God, Faith, Dating & Relati...",Why are relationships so hard? Why do I worry ...,Chloe M. Gooden,['en'],Why We Go Back to Guys We Know Aren't Good For Us,How many times have you gone back to an ex tha...,20.473417,show_2UyVUlVZBrZxOPcNeHO59D,Religion & Spirituality,NaN
11,show_2UyVUlVZBrZxOPcNeHO59D,2pQLJKsfwaUtduKbRl1TrW,Hello and welcome to combos with Chloe. So tod...,0.820536,16028,"Convos With Chloe: God, Faith, Dating & Relati...",Why are relationships so hard? Why do I worry ...,Chloe M. Gooden,['en'],How to Be at Peace with an Unknown Future,I hate not knowing what will happen next in my...,15.770217,show_2UyVUlVZBrZxOPcNeHO59D,Religion & Spirituality,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65346,show_6Ov00yhKCAgR6BLomUF6yY,2fqISbi9SIiBeMTOLf1Rgr,Hello guys and welcome to the Frozen moat podc...,0.809614,27365,Frozen Milk,Welcome to the Frozen Milk podcast! Here we ta...,Cody Larsen,['en'],Doobs Talk Pt. 1,Me and some beautiful goons named Cameron Chri...,29.850800,show_6Ov00yhKCAgR6BLomUF6yY,NaN,NaN
65450,show_6OUisS4N6LZEwpglRdgu4w,6P0Ydyh5JWPPrbUkXS4kFo,Welcome to the sparkling podcast where you wil...,0.836591,18616,Sparklink Podcast,Click play to listen to talented young adults ...,Ashna Reddy,['en'],Episode #1: Improving Memory w/ Humm,Ever wondered if technology could improve our ...,16.199400,show_6OUisS4N6LZEwpglRdgu4w,Education,NaN
65454,show_6o5SbgWOjYKdmButHAQ0Q6,5H3cwiOfAbchuvC1ZRcOPS,"Hello, good day. Welcome to my podcast on this...",0.771048,1440,Charto Podcast,"Well, I’m here to inform you that today is not...",Sixvalo,['en'],Sp 2 Complete Guide to Advertising on Instagram,Instagram advertising is method of paying to p...,2.236467,show_6o5SbgWOjYKdmButHAQ0Q6,NaN,NaN
65455,show_6o5SbgWOjYKdmButHAQ0Q6,0oTaZBWOdMXw2JOrNHwlgg,"Good morning, welcome to my podcast. Happy Sun...",0.823880,2712,Charto Podcast,"Well, I’m here to inform you that today is not...",Sixvalo,['en'],Sp 1 Complete Guide to Advertising on Instagram,So you’re running paid search and display ads ...,4.798783,show_6o5SbgWOjYKdmButHAQ0Q6,NaN,NaN


### Correct mistake in word count from directory walk

In [10]:
# word_count was actually char count 
full_dataset = full_dataset.rename(columns={"word_count": "char_count"})

# count words
full_dataset["word_count"] = full_dataset.transcript.apply(lambda x: len(x.split(" ")))

In [11]:
languages_not_keep = ["['nl-BE']","['hi']","['id']","['ml']","['ms']","['pt']","['en-JM']","['en-IN']","['ga']","['ta']","['es']"]
languages = list(set(full_dataset.language.unique()) - set(languages_not_keep))

In [12]:
full_dataset = full_dataset[full_dataset['language'].isin(languages)]
print(full_dataset.shape)
full_dataset.language.value_counts()

(105228, 16)


['en']       79452
['en-US']    20264
['en-GB']     2335
['en-AU']     1377
['en-CA']      962
['en-PH']      368
['en-IE']      243
['en-NZ']      141
['en-ZA']       86
Name: language, dtype: int64

In [13]:
# Save the DataFrame as a gzip-compressed CSV file
full_dataset.to_csv('transcripts_dataset_final.csv.gz', compression='gzip', index=False)

In [14]:
sample = full_dataset.sample(15000)
sample.to_csv('transcripts_sample.csv.gz', compression='gzip', index=False)